# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. 

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using.

In [1]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

image_name = '<YOUR_DOCKER_IMAGE>' # 'fboylu/kerastf-gpu' Feel free to use this image if you want to 
                                   # skip creating your own container
selected_subscription = "'<YOUR_SUBSCRIPTION>'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

In [1]:
resource_group = "fbaksnaslrg" # Feel free to modify these
aks_name = "fbAKSnaslCluster"
location = "eastus"

image_name = "fboylu/kerasnasl-gpu" 
selected_subscription = "'Team Danielle Internal'"

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [6]:
!az login -o table

In [7]:
!az account set --subscription $selected_subscription

In [2]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "isDefault": true,
  "name": "Team Danielle Internal",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "fboylu@microsoft.com",
    "type": "user"
  }
}


You will also need to register the container service resources on your subscription if you haven't already done so.

In [9]:
!az provider register -n Microsoft.ContainerService

In [10]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [3]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fbaksnaslrg",
  "location": "eastus",
  "managedBy": null,
  "name": "fbaksnaslrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster in the resource group we created earlier.

In [4]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_NC6

{- Finished ..principal creation[##################################]  100.0000%
  "agentPoolProfiles": [
    {
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSnaslC-fbaksnaslrg-edf507",
  "fqdn": "fbaksnaslc-fbaksnaslrg-edf507-7218af32.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbaksnaslrg/providers/Microsoft.ContainerService/managedClusters/fbAKSnaslCluster",
  "kubernetesVersion": "1.9.6",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDTZYQFHNstYCR25qtvMrC6baTMS6TobaIRbgd0xOoafDy+2uBk0DMJuhGWoOcrsCnvadp5k/0K8qBRysyhlQGWb6+r8fBunThy+zpTKqdh3W8Q1y5UtKnGwwU1cqGXDOPUIXJYNPJqUKV82

In [5]:
!az aks list -o table

Name              Location    ResourceGroup    KubernetesVersion    ProvisioningState    Fqdn
----------------  ----------  ---------------  -------------------  -------------------  -----------------------------------------------------------
dangdevopsclstr   eastus      dangdevops       1.9.6                Succeeded            devops-3b040ce9.hcp.eastus.azmk8s.io
ddaks             eastus      ddaksdeploy      1.8.11               Failed               ddaks-ddaksdeploy-edf507-eefe283f.hcp.eastus.azmk8s.io
fbAKSClustercpu   eastus      fbakscpurg       1.8.10               Succeeded            fbaksclust-fbakscpurg-edf507-17685694.hcp.eastus.azmk8s.io
fbAKSnaslCluster  eastus      fbaksnaslrg      1.9.6                Succeeded            fbaksnaslc-fbaksnaslrg-edf507-7218af32.hcp.eastus.azmk8s.io
msaksgpu          eastus      msaksrg          1.9.6                Succeeded            msaksgpu-msaksrg-edf507-0eeb29a6.hcp.eastus.azmk8s.io


### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [6]:
!sudo -i az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [7]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbAKSnaslCluster" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [8]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-55237306-0   Ready     agent     14m       v1.9.6


Let's check the pods on our cluster.

In [10]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-79c5db744-97j2l              1/1       Running   2          14m
kube-system   heapster-55f855b47-h8zz5                2/2       Running   0          13m
kube-system   kube-dns-v20-7c556f89c5-c9bzs           3/3       Running   0          14m
kube-system   kube-dns-v20-7c556f89c5-nh8tf           3/3       Running   0          14m
kube-system   kube-proxy-kcsss                        1/1       Running   0          14m
kube-system   kube-svc-redirect-7vvc9                 1/1       Running   0          14m
kube-system   kubernetes-dashboard-546f987686-rx47d   1/1       Running   1          14m
kube-system   tunnelfront-fd46fb4b9-8ld49             1/1       Running   0          14m


## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [11]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "fboylu/kerasnasl-gpu",
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "mountPath":"/usr/local/nvidia",
                              "name": "nvidia",
                          }
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "nvidia",
                      "hostPath":{
                          "path":"/usr/local/nvidia"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [12]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4, sort_keys=True)
        outfile.write('\n\n')

In [13]:
write_json_to_file(app_template, 'az-dl.json')

In [14]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [15]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                            }
                        ],
                        "image": "fboylu/kerasnasl-gpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
             

Next, we will use kubectl create command to deploy our application.

In [16]:
!kubectl create -f az-dl.json

deployment.apps "azure-dl" created
service "azure-dl" created


Let's check if the pod is deployed.

In [26]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-74c4d5d9d8-7fzvw               1/1       Running   0          10m
kube-system   azureproxy-79c5db744-97j2l              1/1       Running   2          26m
kube-system   heapster-55f855b47-h8zz5                2/2       Running   0          24m
kube-system   kube-dns-v20-7c556f89c5-c9bzs           3/3       Running   0          26m
kube-system   kube-dns-v20-7c556f89c5-nh8tf           3/3       Running   0          26m
kube-system   kube-proxy-kcsss                        1/1       Running   0          26m
kube-system   kube-svc-redirect-7vvc9                 1/1       Running   0          26m
kube-system   kubernetes-dashboard-546f987686-rx47d   1/1       Running   1          26m
kube-system   tunnelfront-fd46fb4b9-8ld49             1/1       Running   0          26m


If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [27]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                    SOURCE                                 MESSAGE
28m         28m          1         aks-nodepool1-55237306-0.153559872ca2345b    Node                                     Normal    Starting                  kubelet, aks-nodepool1-55237306-0      Starting kubelet.
27m         28m          8         aks-nodepool1-55237306-0.153559872ebdfb3c    Node                                     Normal    NodeHasSufficientDisk     kubelet, aks-nodepool1-55237306-0      Node aks-nodepool1-55237306-0 status is now: NodeHasSufficientDisk
27m         28m          8         aks-nodepool1-55237306-0.153559872ebe5578    Node                                     Normal    NodeHasSufficientMemory   kubelet, aks-nodepool1-55237306-0      Node aks-nodepool1-55237306-0 status is now: NodeHasSufficientMemory
27m         28m          7         aks-nodepool1-55237

Check the logs for the application pod.

In [28]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [29]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-06-05 19:39:49,581 CRIT Supervisor running as root (no user in config file)
2018-06-05 19:39:49,583 INFO supervisord started with pid 1
2018-06-05 19:39:50,584 INFO spawned: 'program_exit' with pid 9
2018-06-05 19:39:50,586 INFO spawned: 'nginx' with pid 10
2018-06-05 19:39:50,588 INFO spawned: 'gunicorn' with pid 11
2018-06-05 19:39:51,618 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-06-05 19:39:52.002225: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-06-05 19:39:52.233672: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1212] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: e2e1:00:00.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-06-05 19:39:52.233718: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1312] Adding visible gpu devices: 0
2018-06-05 

In [30]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   1         1         1            1           11m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [31]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)        AGE
azure-dl   LoadBalancer   10.0.251.102   137.117.94.64   80:32742/TCP   11m


Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb). Once, we are done with all the notebooks of the tutorial, below instructions can be used to delete the cluster and free resources.

# Scaling

In [36]:
!az aks scale --resource-group=$resource_group --name=$aks_name --node-count 3

{- Finished ..
  "agentPoolProfiles": [
    {
      "count": 3,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSnaslC-fbaksnaslrg-edf507",
  "fqdn": "fbaksnaslc-fbaksnaslrg-edf507-7218af32.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbaksnaslrg/providers/Microsoft.ContainerService/managedClusters/fbAKSnaslCluster",
  "kubernetesVersion": "1.9.6",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDTZYQFHNstYCR25qtvMrC6baTMS6TobaIRbgd0xOoafDy+2uBk0DMJuhGWoOcrsCnvadp5k/0K8qBRysyhlQGWb6+r8fBunThy+zpTKqdh3W8Q1y5UtKnGwwU1cqGXDOPUIXJYNPJqUKV829+MOrZjUynhHgSzDbY2ncGyoT+Farsvm01aGEdDapa+XRl4JAwtN1bb9q+Ii5y+Mk

In [37]:
!kubectl describe nodes

Name:               aks-nodepool1-55237306-0
Roles:              agent
Labels:             agentpool=nodepool1
                    beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/instance-type=Standard_NC6
                    beta.kubernetes.io/os=linux
                    failure-domain.beta.kubernetes.io/region=eastus
                    failure-domain.beta.kubernetes.io/zone=0
                    kubernetes.azure.com/cluster=MC_fbaksnaslrg_fbAKSnaslCluster_eastus
                    kubernetes.io/hostname=aks-nodepool1-55237306-0
                    kubernetes.io/role=agent
                    storageprofile=managed
                    storagetier=Standard_LRS
Annotations:        node.alpha.kubernetes.io/ttl=0
                    volumes.kubernetes.io/controller-managed-attach-detach=true
CreationTimestamp:  Tue, 05 Jun 2018 19:16:15 +0000
Taints:             <none>
Unschedulable:      false
Conditions:
  Type                 Status  LastHeartbeatTime           



Name:               aks-nodepool1-55237306-2
Roles:              agent
Labels:             agentpool=nodepool1
                    beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/instance-type=Standard_NC6
                    beta.kubernetes.io/os=linux
                    failure-domain.beta.kubernetes.io/region=eastus
                    failure-domain.beta.kubernetes.io/zone=0
                    kubernetes.azure.com/cluster=MC_fbaksnaslrg_fbAKSnaslCluster_eastus
                    kubernetes.io/hostname=aks-nodepool1-55237306-2
                    kubernetes.io/role=agent
                    storageprofile=managed
                    storagetier=Standard_LRS
Annotations:        node.alpha.kubernetes.io/ttl=0
                    volumes.kubernetes.io/controller-managed-attach-detach=true
CreationTimestamp:  Wed, 06 Jun 2018 15:21:42 +0000
Taints:             <none>
Unschedulable:      false
Conditions:
  Type                 Status  LastHeartbeatTime         

In [38]:
!kubectl scale --current-replicas=2 --replicas=3 deployment/azure-dl

deployment.extensions "azure-dl" scaled


In [45]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-74c4d5d9d8-7fzvw               1/1       Running   0          21h
default       azure-dl-74c4d5d9d8-mns89               1/1       Running   0          3h
default       azure-dl-74c4d5d9d8-s4qbf               1/1       Running   0          1h
kube-system   azureproxy-79c5db744-97j2l              1/1       Running   2          22h
kube-system   heapster-55f855b47-h8zz5                2/2       Running   0          22h
kube-system   kube-dns-v20-7c556f89c5-c9bzs           3/3       Running   0          22h
kube-system   kube-dns-v20-7c556f89c5-nh8tf           3/3       Running   0          22h
kube-system   kube-proxy-kcsss                        1/1       Running   0          22h
kube-system   kube-proxy-wpzpw                        1/1       Running   0          3h
kube-system   kube-proxy-zlhhs                        1/1       Running   0          2h
kube-system   kube-svc-re

# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [46]:
!kubectl delete -f az-dl.json

deployment.apps "azure-dl" deleted
service "azure-dl" deleted


In [47]:
!az aks delete -n $aks_name -g $resource_group -y

inished ..

In [48]:
!az group delete --name $resource_group -y

inished ..